In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pathlib
import glob
import itertools
import pandas as pd
import seaborn as sns
import pydot

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2M
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.utils import plot_model

from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from mlxtend.plotting import plot_confusion_matrix


In [2]:
train_data_dir = r"D:/experiments/Datasets/ip102_v1.1-002/ip102_v1.1/prepared_data/train_stg/"
valid_data_dir = r"D:/experiments/Datasets/ip102_v1.1-002/ip102_v1.1/prepared_data/val_stg/"
test_data_dir = r"D:/experiments/Datasets/ip102_v1.1-002/ip102_v1.1/prepared_data/test_stg/"

In [3]:
classes = [
        'Adult',
        'Larva',
        'Pupa',
        'Egg'
]

ResNet50

In [17]:
datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    vertical_flip=True,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [18]:
#Without data augmentation
# datagen = ImageDataGenerator()

In [19]:
train_batchs_Res = datagen.flow_from_directory(directory=train_data_dir, target_size=(224,224), classes=classes,class_mode='categorical', batch_size=64)

valid_batchs_Res = datagen.flow_from_directory(directory=valid_data_dir, target_size=(224,224), classes=classes,class_mode='categorical', batch_size=64)

test_batchs_Res = datagen.flow_from_directory(directory=test_data_dir, target_size=(224,224), classes=classes,class_mode='categorical', batch_size=64, shuffle=False)

Found 38400 images belonging to 4 classes.
Found 6352 images belonging to 4 classes.
Found 20047 images belonging to 4 classes.


In [20]:
base_model_Res = ResNet50(include_top = False, weights = 'imagenet',input_shape=(224, 224, 3))

for layer in base_model_Res.layers:
    layer.trainable = True

In [21]:
x = base_model_Res.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)

predictions = Dense(train_batchs_Res.num_classes, activation='softmax')(x)

modelRes = Model(inputs=base_model_Res.input, outputs=predictions)
modelRes.compile(optimizer = Adam(learning_rate=0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [22]:
callbacks = [
        ModelCheckpoint("SavedModels/ResNet50/ResNet50_stg_Adam.h5", verbose=1, save_best_model=True),
        ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.1, verbose=1, min_lr=1e-6),
        EarlyStopping(monitor="val_loss", patience=5, verbose=1)
    ]

In [23]:
history_Res = modelRes.fit(x = train_batchs_Res, 
            steps_per_epoch=train_batchs_Res.samples // 64,
            validation_data = valid_batchs_Res,
            validation_steps=valid_batchs_Res.samples // 64,
            epochs = 500, verbose = 1,
            callbacks=callbacks,
)

Epoch 1/500
600/600 [==============================] - ETA: 0s - loss: 0.1991 - accuracy: 0.9294
Epoch 1: saving model to SavedModels/ResNet50\ResNet50_stg_Adam.h5
600/600 [==============================] - 207s 341ms/step - loss: 0.1991 - accuracy: 0.9294 - val_loss: 0.5413 - val_accuracy: 0.8378 - lr: 1.0000e-04
Epoch 2/500
600/600 [==============================] - ETA: 0s - loss: 0.1217 - accuracy: 0.9583
Epoch 2: saving model to SavedModels/ResNet50\ResNet50_stg_Adam.h5
600/600 [==============================] - 204s 339ms/step - loss: 0.1217 - accuracy: 0.9583 - val_loss: 0.1785 - val_accuracy: 0.9459 - lr: 1.0000e-04
Epoch 3/500
600/600 [==============================] - ETA: 0s - loss: 0.1007 - accuracy: 0.9646
Epoch 3: saving model to SavedModels/ResNet50\ResNet50_stg_Adam.h5
600/600 [==============================] - 204s 340ms/step - loss: 0.1007 - accuracy: 0.9646 - val_loss: 0.1842 - val_accuracy: 0.9426 - lr: 1.0000e-04
Epoch 4/500
600/600 [==============================]

In [24]:
Res_model = tf.keras.models.load_model('SavedModels/ResNet50/ResNet50_stg_Adam.h5')

Res_predictions = Res_model.predict(x=test_batchs_Res, verbose=0)

In [25]:
y_pred_Res = np.argmax(Res_predictions, axis=1)
y_true_Res = test_batchs_Res.classes

accuracy = accuracy_score(y_true_Res, y_pred_Res)
print('Accuracy: %f' % accuracy)

precision = precision_score(y_true_Res, y_pred_Res, average='weighted', zero_division=0)
print('Precision: %f' % precision)

recall = recall_score(y_true_Res, y_pred_Res, average='weighted', zero_division=0)
print('Recall: %f' % recall)

f1 = f1_score(y_true_Res, y_pred_Res, average='weighted', zero_division=0)
print('F1 score: %f' % f1)

print(classification_report(y_true_Res, y_pred_Res, target_names = classes))

Accuracy: 0.963087
Precision: 0.962437
Recall: 0.963087
F1 score: 0.962485
              precision    recall  f1-score   support

       Adult       0.98      0.98      0.98     16252
       Larva       0.90      0.93      0.91      3344
        Pupa       0.81      0.63      0.71       356
         Egg       0.67      0.51      0.57        95

    accuracy                           0.96     20047
   macro avg       0.84      0.76      0.79     20047
weighted avg       0.96      0.96      0.96     20047



In [26]:
#Get the filename including last folder
image_names = [os.path.basename(filename) for filename in test_batchs_Res.filenames]


# Get the image names from the ImageDataGenerator
image_names = test_batchs_Res.filenames

df = pd.DataFrame({'Image Name': image_names, 'Predicted': y_pred_Res, 'Ground Truth': y_true_Res})
df.to_csv('ResNet50_stg_Adam.csv', index=False)

In [ ]:
matRes = confusion_matrix(y_true_Res,y_pred_Res)
plot_confusion_matrix(matRes, figsize=(4,4), class_names=classes, show_normed=True)